# NFL Draft Data Processing

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from scipy import stats

## Data Sources:

### NFL Draft Data:

https://www.reddit.com/r/NFL_Draft/comments/8cxcxj/2018_nfl_draft_data/

https://docs.google.com/spreadsheets/d/1sFcj1W38gR_Bg3BQz8lZSuFrRQr0BDk8K066ctrxrOA/edit#gid=1586587221

### CFB Standings

https://www.sports-reference.com/cfb/years/2018-standings.html

### Data Loading Functions

In [2]:
years = list(range(1999, 2018))
# column rename map to allow for vertical concatenation between years
consistency_rename_map = {
    "W.1":"ConfW",
    "L.1":"ConfL",
    "Pct.1":"ConfPct",
    "AP Post":"AP Rank"
}
# column rename map for formatting
formatting_rename_map = {
    "Pct":"win_pct",
    "W":"wins",
    "L":"losses",
    "Off":"ppg_off",
    "Def":"ppg_def",
    "SRS":"srs",
    "SOS":"sos",
    "AP Pre":"ap_pre",
    "AP Rank":"ap_rank",
    "AP High":"ap_high",
}

def get_cfb_standings():
    data = []
    for year in years:
        pth = "./cfb_standings_{}.csv".format(year)
        year_data = pd.read_csv(pth)
        year_data = year_data.rename(columns=consistency_rename_map)
        year_data["college_year"] = year
        data.append(year_data)
    combined = pd.concat(data)
    return combined.rename(columns=formatting_rename_map).reset_index(drop=True)

In [3]:
def get_draft_data():
    nfl_draft = pd.read_excel("./NFL-Draft-2018.xlsx")
    nfl_draft["college_year"] = nfl_draft["year"] - 1
    return nfl_draft

In [4]:
cfb_standings = get_cfb_standings()

In [5]:
cfb_standings.columns

Index(['Rk', 'School', 'Conf', 'wins', 'losses', 'win_pct', 'ConfW', 'ConfL',
       'ConfPct', 'ppg_off', 'ppg_def', 'srs', 'sos', 'ap_pre', 'ap_high',
       'ap_rank', 'Notes', 'college_year'],
      dtype='object')

In [6]:
nfl_draft = get_draft_data()

In [7]:
nfl_draft.columns

Index(['url', 'pick', 'team', 'year', 'pos', 'player', 'college',
       'missing_combine_vals', 'height_inches', 'weight', 'forty', 'vertical',
       'bench', 'broad', 'threecone', 'shuttle', 'defense.ast.tackles',
       'defense.fum.forced', 'defense.fum.rec', 'defense.fum.tds',
       'defense.fum.yds', 'defense.games', 'defense.int', 'defense.int.td',
       'defense.int.yards', 'defense.loss.tackles', 'defense.pd',
       'defense.sacks', 'defense.seasons', 'defense.solo.tackes',
       'defense.tackles', 'passing.attempts', 'passing.comp.pct',
       'passing.completions', 'passing.games', 'passing.pass.ints',
       'passing.pass.tds', 'passing.pass.yards', 'passing.seasons',
       'receiving.games', 'receiving.rec.td', 'receiving.rec.yards',
       'receiving.receptions', 'receiving.rush.att', 'receiving.rush.td',
       'receiving.rush.yds', 'receiving.scrim.plays', 'receiving.scrim.tds',
       'receiving.scrim.yds', 'receiving.seasons', 'rushing.games',
       'rushing.re

### Rank to Standard Normal Function

In [8]:
def rank_to_normal(data, N):
    pcts = ((N + 1) - data) / (N + 1)
    return stats.norm.ppf(pcts)

### CFB Standings Transformation Functions

In [9]:
power5_confs = ["SEC (West)", "Big 12", "SEC (East)", "Big Ten (East)",
                "ACC (Coastal)", "ACC (Atlantic)", "Pac-12 (South)",
                "Big Ten (West)", "Pac-12 (North)"]
def is_power5(data):
    return data.Conf.isin(power5_confs).astype(int)

In [10]:
confs = [
    'SEC', 'ACC', 'Big Ten', 'Big 12', 'Pac-10',
    'Sun Belt', 'MAC', 'WAC', 'MWC', 'Ind', 'CUSA'
]

def transform_confs(data):
    data["conf"] = data.Conf.str.replace(r" \(.+\)", "")
    data.loc[data.conf == "Pac-12", "conf"] = "Pac-10"
    data.loc[~data.conf.isin(confs), "conf"] = "Other"
    data.conf = data.conf.fillna("Other")
    return data

def encode_confs(data): 
    enc = OneHotEncoder(sparse=False, dtype=int)
    conf_vals = enc.fit_transform(data[["conf"]])
    conf_col_names = list(enc.get_feature_names(["conf"]))
    conf_df = pd.DataFrame(conf_vals, columns=conf_col_names)
    return pd.concat([data, conf_df], axis=1)

def normalize_ap_rank(data):
    data = data.copy()
    for year, grp in data.groupby("college_year"):
        num_teams = len(grp)
        non_ranked = grp.loc[grp["ap_rank"].isnull()]
        never_ranked = grp.loc[grp["ap_high"].isnull()]
        num_ranked = num_teams - len(never_ranked)
        
        non_ranked_rank = 25 + ((num_ranked - 25) / 2)
        grp.loc[non_ranked.index, "ap_rank"] = non_ranked_rank
        
        never_ranked_rank = num_ranked + ((len(never_ranked)) / 2)
        grp.loc[never_ranked.index, "ap_rank"] = never_ranked_rank
        
        data.loc[grp.index, "ap_rank_score"] = rank_to_normal(grp["ap_rank"], num_teams)
        print("{}: \n {} non ranked teams rank = {}, {} never ranked teams rank = {}"
              .format(year, len(non_ranked), non_ranked_rank, len(never_ranked), never_ranked_rank))
    return data

### NFL Draft Data Transformation Functions

In [11]:
college_map = {
    "Southern California":"USC",
    "Pittsburgh":"Pitt",
    "Mississippi":"Ole Miss",
    "TCU":"Texas Christian",
    "Central Florida": "UCF",
    "Southern Miss": "Southern Mississippi",
    "UNLV": "Nevada-Las Vegas",
    "Texas-El Paso":"UTEP",
    "Ohio St.":"Ohio State",
    "Florida St.":"Florida State",
    "North Carolina St.":"North Carolina State",
    "Penn St.":'Penn State',
    "Southern Methodist":'SMU',
    #"Idaho":
    "Miami":"Miami (FL)",
    "Oklahoma St.":"Oklahoma State",
    "San Diego St.":'San Diego State',
    #"Alabam-Birmingham":
    "Washington St.":'Washington State',
    #"Lousiana-Lafeyette":
    "Brigham Young":'BYU',
    "Bowling Green":'Bowling Green State',
    "Louisiana State":"LSU",
}

def rename_colleges(data):
    """Rename colleges so that the merge properly"""
    return data.college.apply(lambda x: college_map.get(x, x))

def get_undrafted_pick(undrafted, last_pick):
    return last_pick + (len(undrafted) / 2)

def normalize_pick(data, undrafted=False):    
    data = data.copy()
    if not undrafted:
        data = data.loc[data.pick < 300].reset_index(drop=True)
    for year, grp in data.groupby("year"):        
        last_pick = grp.loc[grp.pick < 300].pick.max()
        undrafted = grp.loc[grp.pick >= 300]
        data.loc[undrafted.index, "pick"] = get_undrafted_pick(undrafted, last_pick)
        data.loc[grp.index, "pick_score"] = rank_to_normal(grp.pick, last_pick + len(undrafted))
    return data

### Position Filtering Functions

In [12]:
def filter_oline(data):
    """Filter the data to offensive lineman"""
    return data.loc[data.pos.isin(["OT", "OG", "C", "G", "OL"]), :].reset_index(drop=True)

def filter_wrs(data):
    return data.loc[data.pos == "WR", :].reset_index(drop=True)

def filter_rbs(data):
    return data.loc[data.pos == "RB", :].reset_index(drop=True)

### Position Specific Transformations Functions

In [13]:
def handle_nans(data):
    data = data.fillna(data.mean())
    #data = data.dropna()
    return data.reset_index(drop=True)

def normalize_receiving(data):
    data = data.copy()
    # totals
    data["total_seasons"] = data["receiving.seasons"]
    data["total_games"] = data["receiving.games"]
    data["receiving.total_rec"] = data["receiving.receptions"]
    data["receiving.total_plays"] = data["receiving.scrim.plays"]
    data["receiving.total_yds"] = data['receiving.scrim.yds']
    data["receiving.total_rec_yds"] = data["receiving.rec.yards"]
    data["receiving.total_tds"] = data['receiving.scrim.tds']
    data["receiving.total_rec_tds"] = data["receiving.rec.td"]
    # per game stats
    data["receiving.recs_per_game"] = data["receiving.total_rec"] / data["total_games"]
    data["receiving.yds_per_game"] = data["receiving.total_yds"] / data["total_games"]
    data["receiving.tds_per_game"] = data["receiving.total_tds"] / data["total_games"]
    # per reception
    data["receiving.yds_per_rec"] = data["receiving.total_rec_yds"] / data["receiving.receptions"]
    data["receiving.tds_per_rec"] = data["receiving.total_rec_tds"] / data["receiving.receptions"]
    
    return data

def normalize_rushing(data):
    data = data.copy()
    # totals
    data["total_seasons"] = data["rushing.seasons"]
    data["total_games"] = data["rushing.games"]
    data["rushing.total_plays"] = data["rushing.scrim.plays"]
    data["rushing.total_atts"] = data["rushing.rush.att"]
    data["rushing.total_yds"] = data['rushing.scrim.yds']
    data["rushing.total_rush_yds"] = data["rushing.rush.yds"]
    data["rushing.total_rec_yds"] = data["rushing.rec.yards"]
    data["rushing.total_tds"] = data['rushing.scrim.tds']
    data["rushing.total_rush_tds"] = data["rushing.rush.td"]
    data["rushing.total_rec_tds"] = data["rushing.rec.td"]
    data["rushing.total_recs"] = data["rushing.receptions"]
    # per game stats
    data["rushing.atts_per_game"] = data["rushing.total_atts"] / data["total_games"]
    data["rushing.yds_per_game"] = data["rushing.total_yds"] / data["total_games"]
    data["rushing.rush_yds_per_game"] = data["rushing.total_rush_yds"] / data["total_games"]
    data["rushing.rec_yds_per_game"] = data["rushing.total_rec_yds"] / data["total_games"]
    data["rushing.tds_per_game"] = data["rushing.total_tds"] / data["total_games"]
    data["rushing.rush_tds_per_game"] = data["rushing.total_rush_tds"] / data["total_games"]
    data["rushing.rec_tds_per_game"] = data["rushing.total_rec_tds"] / data["total_games"]
    data["rushing.recs_per_game"] = data["rushing.total_recs"] / data["total_games"]
    # per att stats
    data["rushing.rush_yds_per_att"] = data["rushing.total_rush_yds"] / data["rushing.total_atts"]
    data["rushing.rush_tds_per_att"] = data["rushing.total_rush_tds"] / data["rushing.total_atts"]
    # per rec stats
    data["rushing.rec_yds_per_rec"] = data["rushing.total_rec_yds"] / data["rushing.receptions"]
    data["rushing.rec_tds_per_rec"] = data["rushing.total_rec_tds"] / data["rushing.receptions"]

    return data

def normalize_data(data, cols):
    """Normalize variables using standard scaler"""
    data = data.copy()
    data[cols] = StandardScaler().fit_transform(data[cols].values)  
    return data

### Process Data

In [14]:
cfb_standings = normalize_ap_rank(cfb_standings)

1999: 
 89 non ranked teams rank = 34.5, 70 never ranked teams rank = 79.0
2000: 
 91 non ranked teams rank = 34.0, 73 never ranked teams rank = 79.5
2001: 
 92 non ranked teams rank = 34.5, 73 never ranked teams rank = 80.5
2002: 
 92 non ranked teams rank = 36.5, 69 never ranked teams rank = 82.5
2003: 
 92 non ranked teams rank = 36.0, 70 never ranked teams rank = 82.0
2004: 
 95 non ranked teams rank = 33.5, 78 never ranked teams rank = 81.0
2005: 
 94 non ranked teams rank = 34.0, 76 never ranked teams rank = 81.0
2006: 
 94 non ranked teams rank = 32.0, 80 never ranked teams rank = 79.0
2007: 
 95 non ranked teams rank = 35.5, 74 never ranked teams rank = 83.0
2008: 
 95 non ranked teams rank = 38.0, 69 never ranked teams rank = 85.5
2009: 
 95 non ranked teams rank = 34.5, 76 never ranked teams rank = 82.0
2010: 
 95 non ranked teams rank = 35.5, 74 never ranked teams rank = 83.0
2011: 
 95 non ranked teams rank = 33.0, 79 never ranked teams rank = 80.5
2012: 
 99 non ranked tea

/home/sam/.conda/envs/nfl_draft_analysis/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [15]:
nfl_draft["college"] = rename_colleges(nfl_draft)

In [16]:
data = pd.merge(nfl_draft, cfb_standings,
                how="left", left_on=["college_year", "college"], right_on=["college_year", "School"])

In [17]:
data = (data
        .pipe(normalize_pick, undrafted=True)
        .pipe(transform_confs)
        .pipe(encode_confs))
data["power5"] = is_power5(data)

/home/sam/.conda/envs/nfl_draft_analysis/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1997: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
/home/sam/.conda/envs/nfl_draft_analysis/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1997: RuntimeWarning: invalid value encountered in less
  cond1 = (0 < q) & (q < 1)


### Data Selection Parameters

In [18]:
idx_cols = ['year', 'pos', 'player', 'college', 'team']
measureable_cols = ['height_inches', 'weight', 'forty', 'vertical', 'bench',
                    'broad', 'threecone', 'shuttle']
receiving_cols = ["total_seasons", "total_games", "receiving.total_rec", "receiving.total_plays",
                    "receiving.total_yds", "receiving.total_rec_yds", "receiving.total_tds",
                    "receiving.total_rec_tds", "receiving.recs_per_game",
                    "receiving.yds_per_game", "receiving.tds_per_game", "receiving.yds_per_rec",
                    "receiving.tds_per_rec"]
rushing_cols = ["total_seasons", "total_games", "rushing.total_plays", "rushing.total_atts",
                "rushing.total_yds", "rushing.total_rush_yds", "rushing.total_rec_yds", 
                "rushing.total_tds", "rushing.total_rush_tds", "rushing.total_rec_tds",
                "rushing.total_recs", "rushing.atts_per_game", "rushing.yds_per_game",
                "rushing.rush_yds_per_game", "rushing.rec_yds_per_game", "rushing.tds_per_game",
                "rushing.rush_tds_per_game", "rushing.rec_tds_per_game", "rushing.recs_per_game",
                "rushing.rush_yds_per_att", "rushing.rush_tds_per_att", "rushing.rec_yds_per_rec",
                "rushing.rec_tds_per_rec"]
conf_cols = [x for x in data.columns if x.startswith("conf_")]
team_cols_std = ["win_pct", "wins", "losses", "ppg_off", "ppg_def", "srs", "sos"]
team_cols_no_std = ["ap_rank_score"]
team_cols = team_cols_std + team_cols_no_std + conf_cols
dep_cols = ["pick", "pick_score"]

oline_cols = idx_cols + dep_cols + measureable_cols + team_cols
endog_oline = "pick_std"
exog_oline = measureable_cols + ["ppg_off", "ppg_def", "ap_rank_std", "sos"] + conf_cols
wr_cols = idx_cols + dep_cols + measureable_cols + receiving_cols + team_cols
endog_wrs = "pick_std"
exog_wrs = (measureable_cols +
            ["receiving.games", 'receiving.rec.td', 'receiving.yds_per_rec', 'receiving.receptions'] +
            ["ppg_off", "ppg_def", "ap_rank_std", "sos"] +
            conf_cols)
rb_cols = idx_cols + dep_cols + measureable_cols + rushing_cols + team_cols
endog_rbs = "pick_std"
exog_rbs = (measureable_cols +
            ['rushing.games', 'rushing.yds_per_att', 'rushing.rush.att',
             'rushing.scrim.tds', 'rushing.yds_per_rec', 'rushing.receptions'] +
            ["ppg_off", "ppg_def", "ap_rank_std", "sos"] +
            conf_cols)

### Write cleaned data

In [19]:
data.to_csv("nfl_draft_clean.csv", index=False)

Create Oline Data:

In [20]:
oline = (data
         .pipe(filter_oline)
         .pipe(lambda df: df.loc[:, oline_cols].reset_index(drop=True))
         .pipe(handle_nans)
         .pipe(normalize_data, cols=measureable_cols + team_cols_std)
).set_index(idx_cols).sort_index()
oline.to_csv("./oline_normalized.csv")

Create WRs data:

In [21]:
wrs = (data
       .pipe(filter_wrs)
       .pipe(normalize_receiving)
       .pipe(lambda df: df.loc[:, wr_cols].reset_index(drop=True))
       .pipe(handle_nans)
       .pipe(normalize_data, cols=measureable_cols + receiving_cols + team_cols_std)
).set_index(idx_cols).sort_index()
wrs.to_csv("./wrs_normalized.csv")

Create RBs Data:

In [22]:
rbs = (data
       .pipe(filter_rbs)
       .pipe(normalize_rushing)
       .pipe(lambda df: df.loc[:, rb_cols].reset_index(drop=True))
       .pipe(handle_nans)
       .pipe(normalize_data, cols=measureable_cols + rushing_cols + team_cols_std)
).set_index(idx_cols).sort_index()
rbs.to_csv("./rbs_normalized.csv")

### Data Validation

In [23]:
data[["conf", "ap_rank_score"]].info()

KeyError: "['ap_rank_std'] not in index"

In [ ]:
data.conf.value_counts()

In [ ]:
data.groupby("conf").agg({"pick":["count", "mean", "median"]})

In [ ]:
data.columns

In [ ]:
oline[["pick_std"] + exog_oline].describe().T

In [ ]:
wrs[[endog_wrs] + exog_wrs].describe().T

In [ ]:
rbs[[endog_rbs] + exog_rbs].describe().T

In [ ]:
data.pick_std.hist(bins=50)

In [ ]:
data.ap_rank_std.hist(bins=50)

In [ ]:
data.conf.value_counts()

In [ ]:
oline.win_pct.hist()

Top college counts not being merged

In [ ]:
(data.loc[~data.college.isin(cfb_standings.School)].college.value_counts().head(20))

Players with colleges not getting merged:

In [ ]:
data.loc[data.wins.isnull()].head(10)